# Configuration

In [1]:
%matplotlib inline

import sklearn.cluster as skcl;
import pandas as pd;
import matplotlib.pyplot as plt
import feature_normalization as fn

## Read Data

In [2]:
features = pd.read_csv( filepath_or_buffer="features.txt" , sep="\t" )
features = features.query( 'longest_stop_s >= 0' )
print( features.head( 3 ) );

      agentId  longest_stop_s  longest_trip_m  total_stop_s   total_trip_m
0  1000000700         42572.0    85823.402291       71995.0  260486.880229
2  1000001200         46706.0    29981.333807       82621.0   59067.391061
3  1000002000         72688.0    64868.908859       80616.0  111247.858032


## Plot data

In [5]:
feature_names = features.columns.values

# start at 1 because 0 is agent id
#for i in range(1, len( feature_names ) ):
#    #print( "index i=" , i )
#    for j in range( i + 1 , len( feature_names ) ):
#        #print( "index j=" , j )
#        feature1 = feature_names[ i ]
#        feature2 = feature_names[ j ]
#        plt.scatter( features[ feature1 ] , features[ feature2 ] )
#        plt.xlabel( feature1 )
#        plt.ylabel( feature2 )
#        plt.show()

## Normalize features


In [4]:
norm_features = fn.normalize_features( features )

print( norm_features.head() )


      agentId  longest_stop_s  longest_trip_m  total_stop_s  total_trip_m
0  1000000700       -0.000004        0.000014     -0.000020      0.000021
2  1000001200       -0.000003        0.000003     -0.000002      0.000002
3  1000002000        0.000002        0.000010     -0.000006      0.000007
5  1000002500        0.000004       -0.000002      0.000002     -0.000002
6  1000004400        0.000005        0.000001      0.000002     -0.000001


## Train clustering algorithm

### K Means

In [7]:
kmeans = skcl.KMeans(n_clusters=8)

kmeans.fit(norm_features)
pred = kmeans.predict( norm_features )

# start at 1 because 0 is agent id
#for i in range(1, len( feature_names ) ):
#    #print( "index i=" , i )
#    for j in range( i + 1 , len( feature_names ) ):
#        #print( "index j=" , j )
#        feature1 = feature_names[ i ]
#        feature2 = feature_names[ j ]
#        plt.scatter( features[ feature1 ] , features[ feature2 ] , c=pred )
#        plt.xlabel( feature1 )
#        plt.ylabel( feature2 )
#        plt.show()

## Meaningful clustering

Clustering based on meaningful boundaries.

First define some charging times. From https://www.clippercreek.com/wp-content/uploads/2016/04/TIME-TO-CHARGE-20170706_FINAL-LOW-RES.jpg, full charging times range from 2 to 70 hours depending on vehicle and charging station.

In [ ]:
charge_time_thresholds = [ x for i ]